In [1]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
import torch
import oracledb
import os
import pickle
from sklearn.decomposition import PCA
import umap.umap_ as umap
import hdbscan

from collections import Counter
from itertools import chain


In [ ]:
query = 'SELECT * FROM USERS_WANG_YUHANG."biotechnology_title_abstract"'
connection = oracledb.connect(user="", password="", host="",port=,service_name="")
df_tech = pd.read_sql(query, connection)
df_tech=df_tech[['appln_id','appln_title']]
df_tech

In [ ]:
df_tech=df_tech.rename({'appln_title':'title'},axis=1)
df_tech

In [ ]:
# df_tech=pd.read_csv('data/biotechnology_title_abstract.csv',usecols=['appln_id','appln_title'])
# df_tech=df_tech.rename({'appln_title':'title'},axis=1)
# df_tech

In [3]:
df_journal_list=pd.read_excel('data/extlistMarch2023.xlsx',sheet_name='Scopus Sources October 2022')
df_journal_list=df_journal_list[df_journal_list['1300\nBiochemistry, Genetics and Molecular Biology'].notnull()|
               df_journal_list['2400\nImmunology and Microbiology'].notnull()|
                df_journal_list['2700\nMedicine'].notnull()|
                df_journal_list['3000\nPharmacology, Toxicology and Pharmaceutics'].notnull()
               ]
df_journal_list

,Sourcerecord ID,Source Title (Medline-sourced journals are indicated in Green),Print-ISSN,E-ISSN,Active or Inactive,Coverage,Titles discontinued by Scopus due to quality issues,Article language in source (three-letter ISO language codes),2021\nCiteScore,Medline-sourced Title? (see more info under separate tab),...,2700\nMedicine,2800\nNeuroscience,2900\nNursing,"3000\nPharmacology, Toxicology and Pharmaceutics",3100\nPhysics and Astronomy,3200\nPsychology,3300\nSocial Sciences,3400\nVeterinary,3500\nDentistry,3600\nHealth Professions
2,21100447128,3 Biotech,2190572X,21905738,Active,"2014-ongoing, 2012",NaN,ENG,4.3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,21100932761,3D Printing in Medicine,NaN,23656271,Active,2019-ongoing,NaN,ENG,NaN,NaN,...,Medicine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,21100399164,A&A case reports,23257237,NaN,Inactive,2015-2017,NaN,ENG,NaN,Medline-sourced,...,Medicine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,21100881366,A&A practice,NaN,25753126,Active,2018-ongoing,NaN,ENG,0.9,Medline-sourced,...,Medicine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,19600162043,A.M.A. American Journal of Diseases of Children,00968994,NaN,Inactive,1945-1955,NaN,NaN,NaN,NaN,...,Medicine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43389,79328,Zucker,0044538X,NaN,Inactive,1973-1977,NaN,NaN,NaN,NaN,...,Medicine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43391,61333,Zurcher medizingeschichtliche Abhandlungen,05144264,NaN,Inactive,"1999-2002, 1994-1996",NaN,NaN,NaN,NaN,...,Medicine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43395,24168,ZWR,0044166X,NaN,Inactive,1970-1991,NaN,NaN,NaN,NaN,...,Medicine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43397,19073,Zygote,09671994,14698730,Active,1993-ongoing,NaN,ENG,3.3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df_journal_list.to_csv('data/df_journal_list.csv',encoding='utf-8-sig')

In [ ]:
df_journal_list[['Sourcerecord ID']].to_csv('data/Sourcerecord_ID.csv')

In [ ]:
df_sci=pd.read_csv('data/Article.csv',usecols=['eid','doi','title','source_id'],engine='python')
df_sci=df_sci[df_sci['source_id'].isin(df_journal_list['Sourcerecord ID'])]
df_sci

In [ ]:
df_record=pd.concat([df_tech, df_sci],axis=0)
df_record

In [ ]:
del df_tech
del df_sci

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('/data/huggingface/hub/sentence-transformers_all-MiniLM-L6-v2')

In [ ]:
title_embedding = model.encode(df_record['title'].astype(str).tolist(),convert_to_numpy=True,show_progress_bar=True,batch_size=256)

In [ ]:
df_record['title_embedding']=list(title_embedding)

In [ ]:
df_record2=df_record[~((df_record['doi'].isnull())&(df_record['appln_id'].isnull()))].reset_index(drop=True)
df_record2

In [ ]:
len(df_record2['appln_id'].unique())

In [ ]:
df_record2.to_pickle('data/df_record.pickle')

# 加载数据

In [ ]:
df_record=pd.read_pickle('data/df_record.pickle')
df_record

In [ ]:
pca = PCA(n_components=50,random_state=42)
pca_embeddings=pca.fit_transform(df_record['title_embedding'].tolist())

In [ ]:
with open('pca_embeddings.pickle', 'wb') as f:
    pickle.dump(pca_embeddings, f)

In [ ]:
del df_record

In [ ]:
with open('pca_embeddings.pickle', 'rb') as f:
    pca_embeddings = pickle.load(f)

In [ ]:
# 方案1 社区聚类  
# 实测两天蛋都没跑出来
# clusters = util.community_detection(torch.tensor(pca_embeddings), min_community_size=25, threshold=0.75, batch_size=512)

In [ ]:
# 方案2 umap再降维 hdbsacn聚类
clusterable_embedding = umap.UMAP(
    n_neighbors=30,
    min_dist=0.0,
    n_components=10,
    random_state=42,
).fit_transform(pca_embeddings)

In [ ]:
with open('clusterable_embedding.pickle', 'wb') as f:
    pickle.dump(clusterable_embedding, f)

In [2]:
with open('clusterable_embedding.pickle', 'rb') as f:
    clusterable_embedding = pickle.load(f)

In [ ]:
labels = hdbscan.HDBSCAN(
    min_samples=10,
    min_cluster_size=100, #原先是500
).fit_predict(clusterable_embedding)

In [ ]:
with open('labels_cluster_size_100.pickle', 'wb') as f:
    pickle.dump(labels, f)

In [ ]:
Counter(labels).most_common(n=100)

In [ ]:
text = "Using insulin to treat liver cirrhosis"

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM,AutoModelForTokenClassification
import torch

# 聚类可视化

In [2]:
with open('labels.pickle', 'rb') as f:
    labels=pickle.load( f)

In [3]:
with open('clusterable_embedding.pickle', 'rb') as f:
    clusterable_embedding = pickle.load(f)

In [ ]:
# 方案2 umap再降维 hdbsacn聚类
plot_embedding = umap.UMAP(
    n_neighbors=30,
    min_dist=0.0,
    n_components=2,
    random_state=42,
).fit_transform(clusterable_embedding)

/home/wangyuhang/.local/lib/python3.10/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


In [ ]:
with open('plot_embedding.pickle', 'wb') as f:
    pickle.dump(plot_embedding, f)